In [2]:
import os
import pandas as pd
import numpy as np

# Definição de parâmetros
num_nodes = 100000  # quantidade de nós
num_edges = 200000  # quantidade de arestas

edges = []
label_0_count = 0
label_1_count = 0
target_per_label = num_edges // 2

while len(edges) < num_edges:
    src = np.random.randint(0, num_nodes)
    dest = np.random.randint(0, num_nodes)
    if src == dest:  # evitar laços
        continue
    
    # Decidir qual label gerar para manter equilíbrio
    if label_0_count >= target_per_label:
        target_label = 1
    elif label_1_count >= target_per_label:
        target_label = 0
    else:
        target_label = np.random.randint(0, 2)
    
    # Gerar metricas baseadas no target_label
    if target_label == 1:  # aresta boa
        rssi = np.random.uniform(-65, -30)    # sinal mais forte
        etx = np.random.uniform(1, 4)         # menos retransmissões
        delay = np.random.uniform(1, 40)      # menor delay
        busy = np.random.uniform(0, 0.4)      # menos ocupado
    else:  # aresta ruim
        rssi = np.random.uniform(-90, -55)    # sinal mais fraco
        etx = np.random.uniform(3, 10)        # mais retransmissões
        delay = np.random.uniform(25, 100)    # maior delay
        busy = np.random.uniform(0.25, 1)     # mais ocupado
    
    # Adicionar alguma variação para não ficar muito determinístico
    noise_factor = 0.15
    if np.random.random() < noise_factor:
        # Ocasionalmente criar uma exceção para adicionar realismo
        if target_label == 1:
            # Tornar algumas metricas piores
            if np.random.random() < 0.5:
                rssi = np.random.uniform(-80, -60)
            if np.random.random() < 0.5:
                etx = np.random.uniform(2, 6)
        else:
            # Tornar algumas metricas melhores
            if np.random.random() < 0.5:
                rssi = np.random.uniform(-70, -40)
            if np.random.random() < 0.5:
                delay = np.random.uniform(5, 50)
    
    label = target_label
    
    if label == 0:
        label_0_count += 1
    else:
        label_1_count += 1
    
    edges.append([src, dest, rssi, etx, delay, busy, label])

# ====== Salvar dataset ======
df = pd.DataFrame(edges, columns=["src", "dest", "rssi", "etx", "delay", "busy_fraction", "label"])
os.makedirs("../datasets", exist_ok=True)
df.to_csv("../datasets/edges.csv", index=False)

print("Dataset salvo em ../datasets/edges.csv")
print(df.head())
print(f"\nDistribuição dos labels:")
print(df['label'].value_counts())
print("\n")

Dataset salvo em ../datasets/edges.csv
     src   dest       rssi       etx      delay  busy_fraction  label
0  35273  38059 -77.022996  4.204539  91.065363       0.549697      0
1  14440  67232 -50.391060  3.893108  13.263134       0.154174      1
2   6383  35306 -38.231781  3.843186  18.191462       0.095562      1
3  42436  89557 -60.143296  8.585158  27.757753       0.450841      0
4  41360  43895 -80.796139  4.521404  33.785629       0.989980      0

Distribuição dos labels:
label
0    100000
1    100000
Name: count, dtype: int64


